In [1]:
## !pip install pydrive
## !pip install textgrid
## !pip install librosa

In [2]:
import os
import glob
import json
import crepe
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pydub import AudioSegment
from textgrid import TextGrid, IntervalTier, Interval
from codecarbon import EmissionsTracker

In [3]:
output_directory = "data_out/"
audio_directory = "audio/"
text_grid_directory = "textgrid/"
json_directory = "json/"

tracker = EmissionsTracker()

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
if not os.path.exists(output_directory + audio_directory):
    os.makedirs(output_directory + audio_directory)
if not os.path.exists(output_directory + text_grid_directory):
    os.makedirs(output_directory + text_grid_directory)
if not os.path.exists(output_directory + json_directory):
    os.makedirs(output_directory + json_directory)

[codecarbon WARNING @ 20:16:58] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 20:16:58] [setup] RAM Tracking...
[codecarbon INFO @ 20:16:58] [setup] GPU Tracking...
[codecarbon INFO @ 20:16:58] No GPU found.
[codecarbon INFO @ 20:16:58] [setup] CPU Tracking...
[codecarbon WARNING @ 20:17:00] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 20:17:01] CPU Model on constant consumption mode: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
[codecarbon INFO @ 20:17:01] >>> Tracker's metadata:
[codecarbon INFO @ 20:17:01]   Platform system: macOS-14.4.1-x86_64-i386-64bit
[codecarbon INFO @ 20:17:01]   Python version: 3.11.4
[codecarbon INFO @ 20:17:01]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 20:17:01]   Available RAM : 32.000 GB
[codecarbon INFO @ 20:17:01]   CPU count: 16
[codecarbon INFO @ 20:17:01]   CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
[codecarbon INFO @ 20:17:01]   GPU count: None
[codecarbon INFO @ 20:17:

In [4]:
def get_pitches_cr(file, confidence_floor=0):
    sr, audio = wavfile.read(file)
    time, frequency, confidence, _ = crepe.predict(audio, sr, viterbi=True)
    new_freq = []
    new_time = []
    for t, f, c in zip(time, frequency, confidence):
        if c > confidence_floor:
            new_freq.append(f)
            new_time.append(t)
    return new_freq, new_time

In [5]:
def get_intervals(file):
    grid = TextGrid.fromFile(file)
    word_tier = grid[0]
    labels = []
    durations = []
    for interval in word_tier:
            duration = interval.maxTime - interval.minTime
            durations.append(duration)
            labels.append(interval.mark)
    return labels, durations

In [6]:
def graph_audios(data):
    for i in range(len(data)):
        pitches = data[i]['pitches']
        pitch_int = data[i]['pitch_int']

        labels = data[i]['labels']
        label_int = data[i]['label_int']

        max_pitch_int = max(pitch_int)

        fig, ax1 = plt.subplots(figsize=(10, 6))

        ax1.scatter(pitch_int, pitches, color='blue', alpha=0.5)
        ax1.set_title('Pitch vs. Pitch Interval with Segmented Labels')
        ax1.set_xlabel('Seconds')
        ax1.set_ylabel('Pitch (Hz)')
        ax1.set_ylim(0, max(pitches) + 20)
        ax1.set_xlim(0, max_pitch_int) 
        ax1.grid(True, which='both', linestyle='--', linewidth=0.5)

        colors = plt.cm.rainbow(np.linspace(0, 1, len(labels)))

        ax2 = fig.add_axes([0.125, 0.1, 0.775, 0.2])


        ax2.get_yaxis().set_visible(False)
        ax2.set_xlim(ax1.get_xlim())

        start_points = [0] + list(np.cumsum(label_int)[:-1])

        for i, (label, start, interval) in enumerate(zip(labels, start_points, label_int)):
            rect = mpatches.Rectangle((start, 0), interval, 1, color=colors[i])
            ax2.add_patch(rect)
            ax2.text(start + interval / 2, 0.5, label, ha='center', va='center')

        plt.show()

In [7]:
def extract_segments(wav_path, tg_path, pause_threshold=1000, buffer_time=500):
    audio = AudioSegment.from_wav(wav_path)

    tg = TextGrid.fromFile(tg_path)

    base_path = wav_path.split('.')[0]

    base_path = base_path.replace('data_segmented/', '')

    tier = tg[1]

    last_end = 0
    segment_index = 0
    total_duration = len(audio)
    all_audio = []
    for interval in tier:
        if interval.mark == "#" and interval.duration() > (pause_threshold/1000):

            start_time = max(0, int((last_end * 1000) - buffer_time))
            end_time = min(total_duration, int((interval.minTime * 1000) + buffer_time))


            segment = audio[start_time:end_time]

            segment_name = f"{base_path}_{segment_index}.wav"
            segment.export(os.path.join(str(output_directory + audio_directory), segment_name), format="wav")

            pitches = get_pitches_cr(os.path.join(str(output_directory + audio_directory), segment_name), 0.50)

            temp_dict = {
                "filename": f"{base_path}_{segment_index}",
                "pitches": pitches[0],
                "pitch_int": pitches[1],
                "audio_url": f"/file_processing/{output_directory + audio_directory}{segment_name}",
                "textgrid_url": f"/file_processing/{output_directory + text_grid_directory}{base_path}_{segment_index}.TextGrid",
            }

            tg_segment = TextGrid()
            new_tier = IntervalTier(name=tier.name, minTime=0, maxTime=(end_time-start_time)/1000.0)
            for intv in tier.intervals:
                intv_start = intv.minTime * 1000
                intv_end = intv.maxTime * 1000
                if intv_end > start_time and intv_start < end_time:
                    adjusted_start = max(intv_start - start_time, 0) / 1000.0
                    adjusted_end = min(intv_end - start_time, end_time - start_time) / 1000.0
                    new_interval = Interval(adjusted_start, adjusted_end, intv.mark)
                    new_tier.addInterval(new_interval)
            tg_segment.append(new_tier)
            tg_segment_name = f"{base_path}_{segment_index}.TextGrid"
            tg_segment.write(os.path.join(str(output_directory + text_grid_directory), tg_segment_name))

            intervals = get_intervals(os.path.join(str(output_directory + text_grid_directory), tg_segment_name))
            temp_dict['labels'] = intervals[0]
            temp_dict['label_int'] = intervals[1]

            # Save json
            json_file_path = os.path.join(str(output_directory + json_directory), f"{base_path}_{segment_index}.json")
            with open(json_file_path, 'w') as json_file:
                json.dump(temp_dict, json_file)

            all_audio.append(temp_dict)

            segment_index += 1
            last_end = interval.maxTime

    # graph_audios(all_audio)

In [8]:
tracker.start()
for tg in glob.glob("data_segmented/*.textgrid"):
    tg_file = tg
    wav_file = tg.split('.')[0] + '.wav'
    extract_segments(wav_file, tg_file, pause_threshold=1000, buffer_time=250)

tracker.stop()


2024-05-22 20:17:05.920106: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 199ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 189ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 180ms/step


[codecarbon INFO @ 20:17:17] Energy consumed for RAM : 0.000050 kWh. RAM Power : 12.0 W
[codecarbon INFO @ 20:17:17] Energy consumed for all CPUs : 0.000094 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 20:17:17] 0.000144 kWh of electricity used since the beginning.


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 197ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 167ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 188ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 192ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 184ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 218ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 211ms/step


[codecarbon INFO @ 20:17:32] Energy consumed for RAM : 0.000100 kWh. RAM Power : 12.0 W
[codecarbon INFO @ 20:17:32] Energy consumed for all CPUs : 0.000187 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 20:17:32] 0.000287 kWh of electricity used since the beginning.


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 386ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 390ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 7s 482ms/step
1/5 ━━━━━━━━━━━━━━━━━━━━ 1s 402ms/step

[codecarbon INFO @ 20:51:18] Energy consumed for RAM : 0.006847 kWh. RAM Power : 12.0 W
[codecarbon INFO @ 20:51:18] Energy consumed for all CPUs : 0.012842 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 20:51:18] 0.019688 kWh of electricity used since the beginning.


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 415ms/step


[codecarbon INFO @ 20:51:19] Energy consumed for RAM : 0.006852 kWh. RAM Power : 12.0 W
[codecarbon INFO @ 20:51:19] Energy consumed for all CPUs : 0.012852 kWh. Total CPU Power : 22.5 W
[codecarbon INFO @ 20:51:19] 0.019704 kWh of electricity used since the beginning.
/Users/tunji/.local/share/virtualenvs/prosody-visualization-T3L9LaJv/lib/python3.11/site-packages/codecarbon/output_methods/file.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(data.values)])])


0.0016724309927375477